In [1]:
import xlwings as xw
import pandas as pd
from src.parsing import Splitter

In [2]:
sheet_taxation_list = xw.sheets['Ведомость']
taxation_list_df = sheet_taxation_list.range('A1').expand().options(pd.DataFrame, header=1).value
taxation_list_df = taxation_list_df[['Номер точки', 'Наименование', 'Количество', 'Высота', 'Толщина', 'Состояние', 'Кустарник']]

taxation_list_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник
Индекс,,,,,,,
0,1,береза,14 м2,1,0.1,Хорошее,0.0
1,2,ольха,14 м2,2.5,0.08,Хорошее,0.0
2,"3,4",Ольха серая,2,"5,0.6","0.04,0.7",Хорошее,0.0
3,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее,0.0
4,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее,0.0
5,7,береза,1,2.5,0.1,Хорошее,0.0
6,8,береза,72 м2,1.5,0.2,Хорошее,0.0
7,9,ольха,72 м2,2,0.06,Хорошее,0.0
8,10,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее,0.0


In [3]:
from src.parsing import Templates, Parser
import re


def orm_from_taxation_list_item(series: pd.Series) -> list[dict]:
    match_trunk = re.search(Templates.TRUNKS, series['Количество'])
    match_contour = re.search(Templates.CONTOUR, series['Количество'])
    match_line = re.search(Templates.LINE, series['Количество'])
    if not match_contour and not match_line and not match_trunk:
        split_numbers = Splitter.number(series['Номер точки'])
        split_height = Splitter.size(series['Высота'])
        split_diameter = Splitter.size(series['Толщина'])
        split_quality = Splitter.quality(series['Состояние'])
        is_stump = Parser.identification_stump(series['Высота'], series['Толщина'], bool(series['Кустарник']))
        if len(split_numbers) == 1:
            if (not is_stump or "пень" in series['Наименование'].lower()) and len(split_quality) == 1:
                return [series.to_dict()]
        else:
            if (not is_stump or "пень" in series['Наименование'].lower()) and len(split_quality) == 1:
                return [series.to_dict()]
            if len(split_height) == 1:
                split_height = split_height * int(series['Количество'])
            if len(split_diameter) == 1:
                split_diameter = split_diameter * int(series['Количество'])
            if len(split_quality) == 1:
                split_quality = split_quality * int(series['Количество'])
            series_data = []
            for idx in range(len(split_numbers)):
                if "пень" not in series['Наименование'].lower():
                    is_stump = Parser.identification_stump(split_height[idx], split_diameter[idx], bool(series['Кустарник']))
                    name = series['Наименование']+" (пень)" if is_stump else series['Наименование']
                else:
                    name = series['Наименование']
                series_data.append({
                    'Номер точки': split_numbers[idx],
                    'Наименование': name,
                    'Количество': 1,
                    'Высота': split_height[idx],
                    'Толщина': split_diameter[idx],
                    'Состояние': split_quality[idx]
                })
            return series_data
    else:
        return [series.to_dict()]

taxation_list_orm = []
for _, series in taxation_list_df.iterrows():
    # if series.to_dict()['Номер точки'] == '3,4':
    taxation_list_orm.extend(orm_from_taxation_list_item(series))

taxation_list_orm_df = pd.DataFrame(taxation_list_orm)

taxation_list_orm_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник
0,1,береза,14 м2,1,0.1,Хорошее,0.0
1,2,ольха,14 м2,2.5,0.08,Хорошее,0.0
2,3,Ольха серая,1,5,0.04,Хорошее,NaN
3,4,Ольха серая (пень),1,0.6,0.7,Хорошее,NaN
4,5,береза,6 стволов,"4,4,4,4,4,4","5,5,5,5,5,5",Хорошее,0.0
5,6,Ива ломкая,54 ствола,"5х14, 4х30, 3х8,2,1","7х3,4х11, 2х40",Хорошее,0.0
6,7,береза,1,2.5,0.1,Хорошее,0.0
7,8,береза,72 м2,1.5,0.2,Хорошее,0.0
8,9,ольха,72 м2,2,0.06,Хорошее,0.0
9,10,Яблоня домашняя,2 ствола,1.5х2,0.04х2,Хорошее,0.0


In [47]:
def f(height, diameter, is_shrub):
    if is_shrub:
        return False
    split_height = Splitter.size(str(height))
    print(split_height)
    split_diameter = Splitter.size(str(diameter))
    print(split_diameter)
    is_stump = False
    for h, d in zip(split_height, split_diameter):
        print(h,d)
        if float(h) < 0.7 and float(d) / float(h) > 0.1:
            is_stump = True
            print('break')
            break
    return is_stump
f('5, 6', '0.04, 0.7', False)

[5, 6]
[0.04, 0.7]
5 0.04
6 0.7


False